<center><h1>Named Entity Recognition</h1></center>
<hr/>

#### Imports

In [1]:
import pandas as pd
import json
import spacy
import numpy as np

#### Data

In [2]:
story = "\n".join(pd.read_table("data/our_data/The Sleeping Beauty In The Woods.txt", header=None)[0])

In [3]:
ann = json.load(open("data/our_data/The Sleeping Beauty In The Woods.json"))

In [4]:
ann["Relationships"]

{'king': [['queen', 1], ['princess', 1], ['fairies', 0], ['old fairy', -1]],
 'queen': [['king', 1], ['princess', 1], ['fairies', 0]],
 'princess': [['king', 1],
  ['queen', 1],
  ['fairies', 1],
  ['old fairy', -1],
  ['old woman', 0],
  ['prince', 1]],
 'fairies': [['king', 1], ['queen', 0], ['old fairy', -1], ['princess', 1]],
 'old fairy': [['king', -1],
  ['queen', -1],
  ['princess', -1],
  ['fairies', -1],
  ['dwarf', 0]],
 'old woman': [['princess', -1]],
 'dwarf': [['old fairy', 0]],
 'mopsey': [['old fairy', 0]],
 'prince': [['countryman', 0], ['princess', 1]],
 'countryman': [['prince', 0]]}

#### Spacy

<p>Using pre-trained Spacy model without fine-tuining:</p>

In [5]:
nlp = spacy.load("en_core_web_sm")

In [6]:
doc = nlp(story)

In [7]:
pred = np.unique([ent.text.lower() for ent in doc.ents if ent.label_ == "PERSON"])

In [8]:
pred

array(['children,--so', 'mopsey', 'queen'], dtype='<U13')

<p>Trying with nouns at specific number of mentions:</p>

In [9]:
nouns = [token.lemma_ for token in doc if token.pos_ == "NOUN"]

In [10]:
nouns[:10]

['time',
 'king',
 'queen',
 'daughter',
 'christening',
 'godmother',
 'fairy',
 'kingdom',
 'gift',
 'custom']

In [11]:
nouns, counts = np.unique(nouns, return_counts=True)

In [13]:
nouns[counts > 3]

array(['end', 'fairy', 'gift', 'gold', 'hand', 'one', 'order', 'palace',
       'room', 'spindle', 'time', 'tower', 'woman', 'year'], dtype='<U19')

<p>Fine-tuning Spacy to see if it improves results:</p>

In [18]:
import spacy
import random
import tqdm
from spacy.training.example import Example

nlp = spacy.load("en_core_web_sm")

TRAIN_DATA = [
    ("The village was afraid of the monster that lurked in the nearby forest.", {"entities": []}),
    ("There was a farmer who had a magical goose that laid golden eggs.", {"entities": [(12, 18, "PERSON")]}),
    ("The wicked stepmother gave Snow White a poisoned apple.", {"entities": [(25, 34, "PERSON")]}),
    ("A prince came to Rapunzel's tower and called out to her.", {"entities": [(2, 8, "PERSON"), (18, 25, "PERSON")]}),
    ("The Emperor announced that he was looking for a new advisor.", {"entities": [(4, 12, "PERSON")]}),
    ("The three billy goats gruff wanted to cross the bridge to get to the other side.", {"entities": []}),
    ("The big bad wolf tried to blow down the three little pigs' houses.", {"entities": [(4, 16, "PERSON")]}),
    ("The beautiful princess fell asleep for a hundred years when she pricked her finger on a spindle.", {"entities": [(4, 21, "PERSON")]}),
    ("A boy named Jack traded his cow for some magic beans.", {"entities": [(7, 11, "PERSON")]}),
    ("The youngest daughter of a poor miller was able to spin straw into gold.", {"entities": [(20, 23, "PERSON"), (27, 32, "PERSON")]}),
    ("The gingerbread man ran away from the old woman, the old man, and the cow.", {"entities": [(22, 33, "PERSON"), (38, 45, "PERSON"), (50, 53, "PERSON")]}),
    ("The little mermaid fell in love with a prince and traded her voice for legs.", {"entities": [(4, 20, "PERSON"), (37, 44, "PERSON")]}),
    ("A wolf dressed up as Little Red Riding Hood's grandmother to trick her.", {"entities": [(2, 6, "PERSON"), (14, 31, "PERSON")]}),
    ("The king promised his daughter's hand in marriage to the man who could solve his riddle.", {"entities": [(4, 7, "PERSON"), (39, 43, "PERSON")]}),
    ("The greedy farmer tried to take all the wheat for himself, but the Little Red Hen had other plans.", {"entities": [(4, 18, "PERSON"), (59, 74, "PERSON")]}),
    ("The fairy godmother gave Cinderella a beautiful dress and glass slippers for the ball.", {"entities": [(4, 18, "PERSON"), (26, 35, "PERSON")]}),
    ("The shoemaker and his wife received help from elves who made shoes for them at night.", {"entities": [(4, 15, "PERSON"), (23, 29, "PERSON"), (51, 55, "PERSON")]}),
    ("A boy named Aladdin found a magic lamp with a genie inside.", {"entities": [(7, 14, "PERSON"), (26, 32, "PERSON")]}),
    ("The ugly duckling grew up to be a beautiful swan.", {"entities": [(4, 17, "PERSON")]}),
    ("The pied piper led all the rats out of town with his magical flute.", {"entities": [(4, 16, "PERSON")]}),
    ("Hansel and Gretel were lost in the woods and found a house made of candy.", {"entities": [(0, 6, "PERSON"), (9, 15, "PERSON")]}),
    ("The evil queen asked the mirror who the fairest of them all was.", {"entities": [(4, 16, "PERSON")]}),
    ("Pinocchio's nose grew every time he told a lie.", {"entities": [(0, 8, "PERSON")]}),
    ("The seven dwarfs helped Snow White defeat the wicked queen.", {"entities": [(4, 16, "PERSON"), (29, 42, "PERSON")]}),
    ("The little match girl lit matches to keep herself warm on a cold winter night.", {"entities": [(4, 20, "PERSON")]}),
    ("Rumpelstiltskin helped a miller's daughter spin straw into gold, but demanded her firstborn child in return.", {"entities": [(0, 14, "PERSON"), (25, 40, "PERSON")]}),
    ("The emperor had no clothes, but nobody wanted to tell him.", {"entities": [(4, 12, "PERSON")]}),
    ("Tom Thumb was so small he could fit in a teacup.", {"entities": [(0, 8, "PERSON")]}),
    ("The little red hen asked for help planting wheat, but none of her friends wanted to help.", {"entities": [(4, 18, "PERSON")]}),
    ("Goldilocks ate the three bears' porridge, slept in their beds, and broke a chair.", {"entities": [(0, 10, "PERSON")]}),
    ("The frog prince could only be turned back into a human if someone kissed him.", {"entities": [(4, 15, "PERSON")]}),
    ("The wolf and the fox tried to outsmart each other, but both ended up losing.", {"entities": [(4, 8, "PERSON"), (13, 16, "PERSON")]}),
    ("The boy who cried wolf eventually lost the trust of his village.", {"entities": [(4, 7, "PERSON")]}),
    ("The little mermaid made a deal with a sea witch to become human and win the prince's heart.", {"entities": [(4, 19, "PERSON"), (43, 50, "PERSON")]}),
    ("The queen of hearts ordered her soldiers to paint the roses red.", {"entities": [(4, 17, "PERSON")]}),
    ("The sorcerer's apprentice used magic to clean the house, but couldn't stop the broom from flooding the room.", {"entities": [(4, 23, "PERSON")]}),
    ("The magic fish granted a poor fisherman's wishes, but his wife's greed led to their downfall.", {"entities": [(16, 32, "PERSON"), (45, 49, "PERSON")]}),
    ("The emperor penguin marched across the Antarctic ice to find a mate.", {"entities": [(4, 17, "PERSON")]}),
    ("The tortoise challenged the hare to a race, and surprisingly won.", {"entities": [(4, 13, "PERSON"), (32, 36, "PERSON")]}),
    ("Robin Hood stole from the rich and gave to the poor.", {"entities": [(0, 10, "PERSON")]}),
    ("The three musketeers were known for their motto: 'All for one, one for all.'", {"entities": [(4, 20, "PERSON")]}),
    ("The wicked witch put a curse on Sleeping Beauty that could only be broken by true love's kiss.", {"entities": [(4, 17, "PERSON"), (34, 48, "PERSON")]}),
    ("Puss in Boots tricked the king into believing his master was wealthy and powerful.", {"entities": [(0, 13, "PERSON"), (47, 52, "PERSON")]}),
    ("The velveteen rabbit became real when a child loved him enough.", {"entities": [(4, 20, "PERSON")]}),
    ("Alice fell down a rabbit hole and found herself in a strange world.", {"entities": [(0, 5, "PERSON")]}),
    ("The genie granted Aladdin three wishes, but warned him to use them wisely.", {"entities": [(4, 8, "PERSON"), (19, 26, "PERSON")]}),
    ("The queen of hearts ordered her soldiers to paint the white roses red.", {"entities": [(4, 18, "PERSON")]}),
    ("The little engine that could chugged up the hill and said, 'I think I can, I think I can.'", {"entities": [(4, 27, "PERSON")]}),
    ("The boy who could talk to animals went on a wild adventure.", {"entities": [(4, 24, "PERSON")]}),
    ("The ugly duckling was shunned by his family but grew up to be a beautiful swan.", {"entities": [(4, 17, "PERSON")]}),
    ("The lion king's uncle plotted to kill him and take over the throne.", {"entities": [(4, 14, "PERSON")]}),
    ("The wizard of Oz sent Dorothy and her dog Toto on a journey to find the Wizard.", {"entities": [(4, 19, "PERSON"), (28, 32, "PERSON")]}),
    ("The nutcracker came to life and led the toys in a battle against the evil Mouse King.", {"entities": [(4, 15, "PERSON"), (61, 71, "PERSON")]}),
    ("The snow queen froze everything in her path, but was eventually defeated by love.", {"entities": [(4, 14, "PERSON")]}),
    ("The boy with the golden arm saved his village from a terrible plague.", {"entities": [(4, 22, "PERSON")]}),
    ("The little mermaid fell in love with a human prince and gave up everything to be with him.", {"entities": [(4, 20, "PERSON"), (34, 41, "PERSON")]}),
    ("The fox and the hound became unlikely friends, despite their differences.", {"entities": [(4, 7, "PERSON"), (12, 17, "PERSON")]}),
    ("The velveteen rabbit learned that love was more important than being real.", {"entities": [(4, 20, "PERSON")]}),
    ("The king summoned the wise old woman for her advice.", {"entities": [(4, 7, "PERSON"), (23, 35, "PERSON")]}),
    ("The queen was furious when she heard the news.", {"entities": [(4, 9, "PERSON")]}),
    ("The wolf sneaked up on Little Red Riding Hood.", {"entities": [(4, 8, "PERSON"), (22, 38, "PERSON")]}),
    ("The prince asked the godmother for help.", {"entities": [(4, 10, "PERSON"), (23, 32, "PERSON")]}),
    ("The stepmother was jealous of Cinderella's beauty.", {"entities": [(4, 14, "PERSON"), (29, 38, "PERSON")]}),
    ("The huntsman saved Snow White from the evil queen.", {"entities": [(4, 12, "PERSON"), (19, 28, "PERSON"), (37, 47, "PERSON")]}),
    ("Aladdin found the genie in the lamp.", {"entities": [(0, 7, "PERSON"), (21, 25, "PERSON"), (29, 33, "PERSON")]}),
    ("The three little pigs built houses out of straw, sticks, and bricks.", {"entities": [(4, 20, "PERSON"), (33, 37, "PERSON"), (39, 44, "PERSON"), (49, 55, "PERSON")]}),
    ("The wicked witch cast a spell on the prince.", {"entities": [(4, 16, "PERSON"), (31, 37, "PERSON")]}),
    ("The emperor had no clothes on.", {"entities": [(4, 12, "PERSON")]}),
    ("The big bad wolf huffed and puffed and blew the house down.", {"entities": [(4, 16, "PERSON"), (27, 38, "PERSON")]}),
    ("The young prince set out on a quest to save the kingdom.", {"entities": [(4, 10, "PERSON")]}),
    ("The queen hired a detective to solve the mystery.", {"entities": [(4, 9, "PERSON"), (21, 30, "PERSON")]}),
    ("The old man told stories to the children in the park.", {"entities": [(4, 12, "PERSON")]}),
    ("The evil stepmother forbade Cinderella from attending the ball.", {"entities": [(4, 19, "PERSON"), (34, 43, "PERSON")]}),
    ("The frog turned into a prince when the princess kissed him.", {"entities": [(4, 7, "PERSON"), (33, 41, "PERSON")]}),
    ("The boy and his dog went on an adventure in the woods.", {"entities": [(0, 3, "PERSON"), (14, 17, "PERSON")]}),
    ("The mother sang a lullaby to her baby.", {"entities": [(4, 10, "PERSON"), (26, 30, "PERSON")]}),
    ("The wise old owl perched on a tree branch.", {"entities": [(4, 16, "PERSON")]}),
    ("The bear wandered into the campsite looking for food.", {"entities": [(4, 8, "PERSON")]}),
    ("The little girl asked the old woman for directions.", {"entities": [(4, 16, "PERSON"), (26, 37, "PERSON")]}),
    ("The prince and princess fell in love and got married.", {"entities": [(4, 10, "PERSON"), (15, 23, "PERSON")]}),
    ("The farmer and his cow won first prize at the county fair.", {"entities": [(4, 10, "PERSON"), (19, 22, "PERSON")]}),
    ("The little boy was scared of the monster under his bed.", {"entities": [(4, 14, "PERSON"), (28, 35, "PERSON")]}),
    ("The dragon terrorized the village and hoarded treasure.", {"entities": [(4, 10, "PERSON")]}),
    ("The mother bear protected her cubs from danger.", {"entities": [(4, 14, "PERSON"), (26, 30, "PERSON")]}),
    ("The old man's parrot repeated everything he said.", {"entities": [(15, 21, "PERSON")]}),
    ("The young princess loved to ride her white horse through the meadows.", {"entities": [(4, 11, "PERSON"), (32, 36, "PERSON")]}),
    ("The knight rode into battle on his trusty steed.", {"entities": [(4, 10, "PERSON"), (28, 33, "PERSON")]}),
    ("The mother hen gathered her chicks under her wings to protect them from the rain.", {"entities": [(4, 14, "PERSON")]}),
    ("The monkey swung from vine to vine in the jungle.", {"entities": [(4, 10, "PERSON")]}),
    ("The prince slayed the dragon and rescued the princess.", {"entities": [(4, 10, "PERSON"), (16, 22, "PERSON"), (36, 44, "PERSON")]}),
    ("The little boy's hamster escaped from its cage and ran around the house.", {"entities": [(15, 22, "PERSON")]}),
    ("The little engine that could kept repeating, 'I think I can, I think I can.'", {"entities": []}),
    ("The town mouse visited his cousin in the city and was amazed by all the sights.", {"entities": [(4, 14, "PERSON"), (41, 45, "PERSON")]}),
    ("A spider spun a web above the pigpen to catch some flies.", {"entities": []}),
    ("The tortoise and the hare had a race, but the hare took a nap and the tortoise won.", {"entities": [(4, 17, "PERSON"), (24, 28, "PERSON")]}),
    ("The cat in the hat caused mischief for two children on a rainy day.", {"entities": [(4, 16, "PERSON")]}),
    ("The gingerbread boy taunted the old woman and the old man until he was caught.", {"entities": [(0, 17, "PERSON"), (22, 35, "PERSON"), (40, 43, "PERSON")]}),
    ("The boy with the red balloon floated away into the sky.", {"entities": [(4, 7, "PERSON")]}),
    ("A group of animals worked together to help a little bird migrate south for the winter.", {"entities": []}),
    ("The little prince traveled from planet to planet and met many interesting characters.", {"entities": [(4, 18, "PERSON")]}),
    ("The ugly duckling was mocked by the other animals, but later discovered that he was a beautiful swan.", {"entities": [(4, 17, "PERSON")]}),
    ("The selfish giant built a wall to keep children out of his garden, but then regretted it.", {"entities": [(4, 18, "PERSON")]}),
    ("Alice fell down a rabbit hole and entered a strange and magical world.", {"entities": [(0, 5, "PERSON")]}),
    ("The boy who owned a horse named Black Beauty was determined to train him well.", {"entities": [(4, 7, "PERSON")]}),
    ("The lion and the mouse helped each other out of sticky situations.", {"entities": [(4, 8, "PERSON"), (13, 18, "PERSON")]}),
    ("Peter Rabbit and his siblings disobeyed their mother and ventured into Mr. McGregor's garden.", {"entities": [(0, 11, "PERSON")]}),
    ("The little house on the prairie was a cozy home for the Ingalls family.", {"entities": []}),
    ("The owl and the pussycat went to sea in a beautiful pea-green boat.", {"entities": [(4, 7, "PERSON"), (13, 22, "PERSON")]}),
    ("The girl who loved horses had a special bond with her favorite stallion.", {"entities": [(4, 8, "PERSON")]}),
    ("The boy who lived under the stairs discovered he was a wizard and attended Hogwarts School of Witchcraft and Wizardry.", {"entities": [(4, 20, "PERSON")]}),
    ("The brave little toaster went on a journey to find his master and save the world.", {"entities": [(4, 22, "PERSON")]}),
    ("The littlest elf helped Santa Claus prepare for Christmas and saved the day.", {"entities": [(4, 17, "PERSON"), (34, 47, "PERSON")]}),
    
]


# Convert (text, annotation) tuples to Example objects
examples = []
for text, annotations in TRAIN_DATA:
    examples.append(Example.from_dict(nlp.make_doc(text), annotations))

# Fine-tune the model using the new examples
nlp.initialize()
optimizer = nlp.create_optimizer()
for epoch in tqdm.tqdm(range(20)):
    random.shuffle(examples)
    for batch in spacy.util.minibatch(examples, size=16):
        nlp.update(batch, sgd=optimizer)


100%|██████████| 20/20 [00:32<00:00,  1.62s/it]


In [19]:
doc = nlp(story)

In [20]:
pred = np.unique([ent.text.lower() for ent in doc.ents if ent.label_ == "PERSON"])

In [21]:
pred

array(['charmed', 'king', 'know', 'prince', 'princess', 'queen'],
      dtype='<U8')

<p>Training Spacy from scratch (not pre-trained):</p>

In [23]:
import random
import spacy
from spacy.training import Example


# define the model
nlp = spacy.blank("en")
ner = nlp.add_pipe("ner")

# train the model
nlp.begin_training()
for i in tqdm.tqdm(range(100)):
    random.shuffle(TRAIN_DATA)
    for text, annotations in TRAIN_DATA:
        example = Example.from_dict(nlp.make_doc(text), annotations)
        nlp.update([example])

# test the model
doc = nlp(u"Princess Kate is married to Prince William.")
print([(ent.text, ent.label_) for ent in doc.ents])


100%|██████████| 100/100 [12:05<00:00,  7.25s/it]

[]


In [24]:
doc = nlp(story)

In [25]:
pred = np.unique([ent.text.lower() for ent in doc.ents if ent.label_ == "PERSON"])

In [26]:
pred

array([',', 'fairy', 'frightful', 'good', 'good old', 'king', 'king gave',
       'king ordered', 'officers', 'old', 'orders', 'pages', 'palace',
       'prince', 'princess', 'queen', 'said:--', 'short', 'spoon',
       'stables', 'stewards', 'too', 'young prince'], dtype='<U12')